# Clustering: extraindo padrões de dados

**Dataset:** Informações sobre clientes de um banco relacionados aos gastos utilizando cartão de crédito.

**Fonte:** Kaggle - [Credit Card Dataset for Clustering](https://www.kaggle.com/datasets/arjunbhasin2013/ccdata)

* Qual o risco dos clientes atrasarem a fatura do cartão? Baixo, Alto, Médio.
* Qual o comportamento dos clientes com o cartão de crédito? Agrupar clientes com características similares.


# Primeiras visualizações do dataset


* O dataset será baixado diretamente via código.
* Para que esse código seja executado, é necessário ter um cadastro no **Kaggle** e gerar seu próprio **API Token**.
* **Tutorial:** https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/
* No Colab, o dataset baixado estará entre os arquivos temporários, acessíveis na barra lateral esquerda.

### Dicionário

**CUST_ID:** identificador do cliente.

**BALANCE:** saldo que o cliente tem disponível no cartão.

**BALANCE_FREQUENCY:** quão frequente o saldo é alterado (valor entre 0 e 1, com 1 sendo maior frequência e 0 menor frequência).

**PURCHASES:** valor gasto em compras no cartão nos últimos 6 meses.

**ONEOFFPURCHASES:** maior valor gasto à vista.

**INSTALLMENTSPURCHASES:** maior valor gasto com parcelamentos.

**CASHADVANCE:** valor de adiantamento pego pelo cliente.

**PURCHASESFREQUENCY:** quão frequente compras são feitas (valor entre 0 e 1, com 1 sendo maior frequência e 0 menor frequência).

**ONEOFFPURCHASESFREQUENCY:** quão frequente compras são feitas à vista (valor entre 0 e 1, com 1 sendo maior frequência e 0 menor frequência).

**PURCHASESINSTALLMENTSFREQUENCY:** quão frequente compras são feitas com parcelamento (valor entre 0 e 1, com 1 sendo maior frequência e 0 menor frequência).

**CASHADVANCEFREQUENCY:** quão frequente são os pagamentos adiantados.

**CASHADVANCETRX:** número de transações feitas com pagamento adiantado.

**PURCHASESTRX:** quantidade de compras feitas no cartão.

**CREDITLIMIT:** limite do cartão de crédito.

**PAYMENTS:** valor de pagamentos feitos pelo cliente.

**MINIMUM_PAYMENTS:** total de pagamentos mínimos realizados.

**PRCFULLPAYMENT:** porcentagem do pagamento integral pago pelo usuário.

**TENURE:** validade do cartão de crédito, tempo para renovação do contrato.


## Abertura do dataset

In [1]:
# Tutorial: https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/

!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/arjunbhasin2013/ccdata")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tassianaoliveira
Your Kaggle Key: ··········


100%|██████████| 340k/340k [00:00<00:00, 35.6MB/s]

In [2]:
import pandas as pd

In [21]:
dataset = pd.read_csv('/content/ccdata/CC GENERAL.csv')
dataset.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


# Tratamentos dos dados

* A identificação do cliente (CUST_ID) e o tempo de contrato do cartão (TENURE) não são valores relacionados ao comportamento dos consumidores, portanto serão removidos do dataset. 

In [22]:
dataset.drop(columns = ['CUST_ID', 'TENURE'], inplace = True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   BALANCE                           8950 non-null   float64
 1   BALANCE_FREQUENCY                 8950 non-null   float64
 2   PURCHASES                         8950 non-null   float64
 3   ONEOFF_PURCHASES                  8950 non-null   float64
 4   INSTALLMENTS_PURCHASES            8950 non-null   float64
 5   CASH_ADVANCE                      8950 non-null   float64
 6   PURCHASES_FREQUENCY               8950 non-null   float64
 7   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 8   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 9   CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 10  CASH_ADVANCE_TRX                  8950 non-null   int64  
 11  PURCHASES_TRX                     8950 non-null   int64  
 12  CREDIT

* Há valores inválidos ou ausentes.

In [23]:
dataset.isna().sum()

BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
dtype: int64

* O **MINIMUN_PAYMENTS** possui uma quantidade considerável de valores faltantes, **CREDIT_LIMIT** tem apenas um valor faltante.

* Estes valores serão preenchidos com a mediana das colunas. Não será um valor exatamente correto, mas é uma aproximação aceitável nesse caso.

In [24]:
dataset.fillna(dataset.median(), inplace = True)
dataset.isna().sum()

BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
dtype: int64

In [25]:
dataset.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000
mean,1564.474828,0.877271,1003.204834,592.437371,411.067645,978.871112,0.490351,0.202458,0.364437,0.135144,3.248827,14.709832,4494.282473,1733.143852,844.906767,0.153715
std,2081.531879,0.236904,2136.634782,1659.887917,904.338115,2097.163877,0.401371,0.298336,0.397448,0.200121,6.824647,24.857649,3638.646702,2895.063757,2332.792322,0.292499
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.019163,0.000000
25%,128.281915,0.888889,39.635000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,383.276166,170.857654,0.000000
50%,873.385231,1.000000,361.280000,38.000000,89.000000,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,856.901546,312.343947,0.000000
75%,2054.140036,1.000000,1110.130000,577.405000,468.637500,1113.821139,0.916667,0.300000,0.750000,0.222222,4.000000,17.000000,6500.000000,1901.134317,788.713501,0.142857
max,19043.138560,1.000000,49039.570000,40761.250000,22500.000000,47137.211760,1.000000,1.000000,1.000000,1.500000,123.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000


* Os valores presentes no dataset possuem muitas variações em suas amplitudes, por exemplo, as frequências, que são de 0 a 1, os valores relacionados a pagamentos, que chegam a até 76 mil.

* Portanto, os dados serão normalizados, entre 0 e 1.

In [26]:
from sklearn.preprocessing import Normalizer

In [30]:
values = Normalizer().fit_transform(dataset.values)
values

array([[3.93555441e-02, 7.87271593e-04, 9.17958473e-02, ...,
        1.94178127e-01, 1.34239194e-01, 0.00000000e+00],
       [2.93875903e-01, 8.34231560e-05, 0.00000000e+00, ...,
        3.76516684e-01, 9.84037959e-02, 2.03923046e-05],
       [3.10798149e-01, 1.24560965e-04, 9.63068011e-02, ...,
        7.74852335e-02, 7.81351982e-02, 0.00000000e+00],
       ...,
       [2.27733092e-02, 8.11060955e-04, 1.40540698e-01, ...,
        7.90986945e-02, 8.02156174e-02, 2.43318384e-04],
       [2.65257948e-02, 1.64255731e-03, 0.00000000e+00, ...,
        1.03579625e-01, 1.09898221e-01, 4.92767391e-04],
       [1.86406219e-01, 3.33426837e-04, 5.46778061e-01, ...,
        3.15915455e-02, 4.41568390e-02, 0.00000000e+00]])

# Primeira Clusterização (K-Means)



In [31]:
from sklearn.cluster import KMeans

In [33]:
kmeans = KMeans(n_clusters = 5, n_init = 10, max_iter = 300)
y_pred = kmeans.fit_predict(values)